In [1]:
import pandas as pd

# Load the CSV files
teams_df = pd.read_csv('data/2024/FantasyFnG_2024.csv')
race1_df = pd.read_csv('data/2024/140124_scores_groupbyTier.csv')
race2_df = pd.read_csv('data/2024/210124_scores_groupbyTier.csv')

In [2]:
# Create a mapping of skier names to points for each race
race1_points = race1_df.set_index('Name')['Points'].to_dict()
race2_points = race2_df.set_index('Name')['Points'].to_dict()

all_races = [race1_points, race2_points]

In [3]:
# Initialize a list to hold the team scores
team_scores = []
detailed_team_scores = {}

In [4]:
# Iterate over each team
for _, team in teams_df.iterrows():
    team_name = team['Fantasy Team Name']
    team_details = {'Total Points': 0, 'Racers': {}}
    
    # Iterate over each tier column to get the skier's name and add their points from each race
    for tier in range(1, 14):
        skier_name = team[f'Tier{tier}']
        if not pd.isna(skier_name):  # Check if skier name is not NaN
            race1_points_skier = race1_points.get(skier_name, 0)
            race2_points_skier = race2_points.get(skier_name, 0)
            
            # Add points to the team's total score
            team_details['Total Points'] += race1_points_skier + race2_points_skier
            
            # Record the racer's points breakdown
            team_details['Racers'][skier_name] = {
                'Race 1 Points': race1_points_skier,
                'Race 2 Points': race2_points_skier
            }
    
    # Add the team's details to the dictionary
    detailed_team_scores[team_name] = team_details

In [5]:
for team, details in detailed_team_scores.items():
    print(team, "has", details["Total Points"])

Set up to win has 76.0
Rudolph Garmisch Crew has 89.0
rickeybobbie has 94.0
C321A has 95.0
All Fun little Glory has 98.0
Suzanne's Quick Finishers has 99.0
Will's RealGudPros  has 98.0
Red Brick Bruisers has 78.0
Topshelf  has 106.0
Insider Trading has 123.0
KKC's Ringers has 98.0
EKC's Ringers has 110.0
Rut Riders has 112.0
Slalom on Challenge! has 94.0


In [6]:
# Extract total points and team names from detailed_team_scores for the summary page
summary_data = [{'Team Name': team_name, 'Total Points': details['Total Points']}
                for team_name, details in detailed_team_scores.items()]

# Convert the summary data to a dataframe
summary_df = pd.DataFrame(summary_data)

# Sort the teams by 'Total Points' in descending order
summary_df = summary_df.sort_values(by='Total Points', ascending=False).reset_index(drop=True)

In [7]:
# Recreate the Pandas Excel writer
writer = pd.ExcelWriter('data/2024/FantasySkiLeagueResults_Aggregated.xlsx', engine='xlsxwriter')

# Write the summary dataframe to the first sheet
summary_df.to_excel(writer, sheet_name='Summary', index=False)

# Iterate over the detailed scores dictionary and write each team's details to a separate sheet
for team_name, details in detailed_team_scores.items():
    # Convert team_name to string and handle None or NaN values
    team_name_str = 'Unknown Team' if pd.isna(team_name) else str(team_name)
    
    # Create a dataframe from the team's racer details
    racers_df = pd.DataFrame(details['Racers']).T
    racers_df.index.name = 'Racer Name'
    racers_df.reset_index(inplace=True)
    
    # Write the team's dataframe to a sheet named after the team
    racers_df.to_excel(writer, sheet_name=team_name_str[:31], index=False)  # Excel sheet name has max 31 characters

# Close the Pandas Excel writer and save the Excel file
writer.save()